In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [25]:
ppfScoutingData = pd.read_csv('nfl-big-data-bowl-2023/pffScoutingData.csv')
games = pd.read_csv('nfl-big-data-bowl-2023/games.csv')
players = pd.read_csv('nfl-big-data-bowl-2023/players.csv')
plays = pd.read_csv('nfl-big-data-bowl-2023/plays.csv')
week1 = pd.read_csv('nfl-big-data-bowl-2023/week1.csv')
week2 = pd.read_csv('nfl-big-data-bowl-2023/week2.csv')
week3 = pd.read_csv('nfl-big-data-bowl-2023/week3.csv')
week4 = pd.read_csv('nfl-big-data-bowl-2023/week4.csv')
week5 = pd.read_csv('nfl-big-data-bowl-2023/week5.csv')
week6 = pd.read_csv('nfl-big-data-bowl-2023/week6.csv')
week7 = pd.read_csv('nfl-big-data-bowl-2023/week7.csv')
week8 = pd.read_csv('nfl-big-data-bowl-2023/week8.csv')

In [26]:
game_data  = pd.concat([week1,week2,week3,week4,week5,week6,week7,week8], axis=0)
data =  game_data.merge(players, on='nflId', how='inner').merge(plays, on = ['gameId','playId'], how='inner').merge(ppfScoutingData, on = ['gameId','playId','nflId'])


In [39]:
from numpy import arange
from pandas import read_csv
from scipy.optimize import curve_fit
#parabola function
def objective(x, a, b, c):
    return  (a * x**2) + (b*x) + c
#get a b c for the offensive line
#returns previous a b c if their is an error
def get_equation(data,d,e,f):
    try:
        o_line = data[data['officialPosition'].isin(['C', 'T', 'G'])]
        rushers = data[data['pff_role'].isin(["Pass Rush"])]
        #all_players = pd.concat([o_line, rushers])
        all_players =o_line
        x = reflect( o_line['y'].values)
        y = o_line['x'].values
        popt, _ = curve_fit(objective, x, y)
    # summarize the parameter values
        a, b, c = popt
        return a,b,c
    except:
        return d,e,f

#distance between two points
def distance(x, y, x0, y0):
    
    d_x = x - x0
    d_y = y - y0
    dis = np.sqrt( d_x**2 + d_y**2 )
    return dis

def min_distance(x, y,  d_x, d_y, precision=5):
   #distance between line and players
    # compute distance
    d = distance(x, y, d_x, d_y)
    d = np.round(d, precision)
    # find the minima
    glob_min_idxs = np.argwhere(d==np.min(d)).ravel()
    return glob_min_idxs, d
#checks if the parabola has inverted based on frame direction
def invert(a, frameDirection):
    if a > 0 and frameDirection =='right':
        return True
    if a < 0 and frameDirection =='left':
        return True
    return False
#checks for a breakthrough using the distance formula
#Note breakthoughs are defined as more than one yard through parabola
def breakthrough(a,b,c, d_x,d_y,frameDirection):
    crossed = False
    def f(x):
        return (a * x**2) + (b*x) + c
    line_x = np.linspace(5, 45,100)
    line_y = f(line_x)
    for i in range(len(d_x)):
        player_x = d_x[i]
        player_y = d_y[i]
        min_idxs, dis = min_distance(line_x, line_y, player_x,player_y)
        y = (a * (player_x**2)) + (b*player_x) + c
        if (min(dis) > 1):
            #check these
            if(y > player_y and frameDirection == 'right'):
                return True
            if(y < player_y and frameDirection == 'left'):
                return True
    return False
#checks if quarterback has left pocket
def leaves_pocket(y, QBy, frameDirection):
    if y < QBy and frameDirection =='right':
        return True
    if y > QBy and frameDirection =='left':
        return True
    return False
#reflects data points to stop mirror image which is necassiry because all
#points are reflected across y = x
def reflect(arr):
    lst = []
 
    for i in arr:
        diff = abs(i - 26.65)
        if i - 26.65 > 0:
            lst.append(26.65-diff)
        else:
             lst.append(26.65 + diff)
    return lst
        
        
        
        

In [40]:

def good_pocket(a, b, c, frame, frameDirection, is_good_pocket,ball_snapped, first_frame):
    #NEED TO ADD EVENT DATA: QB SNAPPING BALL AND PASS
    #Play doesnt start at frame 0
    #If a QB throws the ball while in the pocket, it should be a safe pocket
    
    # 0 means pocket has collapsed
    # Criteria for Pocket collapse
        #1. Parabola flips
        #2. QB steps out of pocket
        #3. Shield is broken by a sack
    if ball_snapped != 1:
        return 1
    if is_good_pocket != 1:
        return is_good_pocket
    
    defense = frame[frame['officialPosition'].isin(['CB', 'DE', 'DT', 'FS', 'ILB', 'MLB', 'OLB','SS'])]
    d_x =  reflect(defense['y'].values)
    d_y =  defense['x'].values
    
    
    QBx = reflect(frame[frame['officialPosition'].isin(['QB'])].y.values)[0]
    QBy =  frame[frame['officialPosition'].isin(['QB'])].x.values[0]
    x = QBx
    y = (a * x**2) + (b*x) + c
    
    defensive_breakthrough = breakthrough(a,b,c,d_x,d_y,frameDirection)
    inverted = invert(a, frameDirection)
    left_pocket = leaves_pocket(y, QBy, frameDirection)
 
    if inverted:
        return 'invert'
    if left_pocket:
        return 'left_pocket'
    if defensive_breakthrough:
        return "breakthrough"
    return 1

In [1]:
## Iterates through every frame of every play for every game and checks the parabola.
## Keeps a running count of frames where there is a good pocket, when the ball is snapped,
## when the ball is thrown, etc.
##
def additional_column_creation(data):
    gameIds = np.unique(data['gameId'])
    a_list = []
    b_list = []
    c_list = []
    last_good_frame = []
    is_good_frame = []
    frame_from_snap = []
    ball_thrown_frame=[]
    for gameId in gameIds:
        print(gameId)
        this_game = data[data['gameId']==gameId]
        playIds = np.unique(this_game['playId'])
        play_count = 0
        frame_count = 0
        for play in playIds:
            this_play = this_game[this_game['playId']==play]
            play_count+=1
            frameIds = np.unique(this_play['frameId'])
            playDirection = this_play.playDirection.values[0]
            pocketFrame = 1
            is_good_pocket = 1
            ball_snapped = 0
            ball_thrown = 0
            snap_frame = 0
            count = 100
            frames_not_snapped = 0
            was_good_pocket = 0
            first_frame = ''
            a,b,c ='','',''
            events = np.unique(this_play['event'])
            temp = '\t'.join(events)
            res = 'snap' in temp
            if res==False:
                ball_snapped = 1
                first_frame = 1
            for frame in frameIds:
                this_frame = this_play[this_play['frameId']==frame] 
                a,b,c = get_equation(this_frame,a,b,c)
                a_list.append(a)
                b_list.append(b)
                c_list.append(c)
                frame_count +=1
                if(ball_thrown!=1):
                    is_good_pocket = good_pocket(a,b,c,this_frame,playDirection,is_good_pocket,ball_snapped,first_frame)
                event = this_frame['event'].values[0]
                if("snap" in event):
                    ball_snapped = 1
                    first_frame = frame
                if(ball_snapped==0):
                    frames_not_snapped +=1
                if (is_good_pocket == 1): 
                    pocketFrame = frame
                    if(ball_snapped == 1):
                        snap_frame +=1
                    if("forward" in event):
                        was_good_pocket=1
                        ball_thrown=1
                if("forward" in event):
                        ball_thrown = 1     
                if(ball_thrown==1 and was_good_pocket):   
                    pocketFrame = frame
                    if res == True:
                        snap_frame +=1
                if(ball_thrown==1):
                    count = frame - frames_not_snapped
                frame_from_snap.append(snap_frame)
                is_good_frame.append(is_good_pocket)
                last_good_frame.append(pocketFrame)
                ball_thrown_frame.append(count)
    return a_list, b_list, c_list, is_good_frame, last_good_frame, frame_from_snap, ball_thrown_frame

In [42]:
################################################################################################################
#  a  #  b  #  c   is_good_pocket. # last_good_pocket_frame. # diff_between_this_frame_and_last_good_pocket_frame
#                         1                        1.                       0
#                         1                        2.                       0
#                         0                        2.                       1
#
#

#Min (Difference divided by .01 = the time it went bad 

one_game

In [44]:
#gets data on the equation for the line, when the line was a good pocket, a bad pocket, when the snap was, and when the ball is thrown
a,b,c,is_good_pocket,frame_last_good_pocket,frame_from_snap, ball_thrown_frame = additional_column_creation(data)


2021090900
2021091200
2021091201
2021091202
2021091203
2021091204
2021091205
2021091206
2021091207
2021091208
2021091209
2021091210
2021091211
2021091212
2021091213
2021091300
2021091600
2021091900
2021091901
2021091902
2021091903
2021091904
2021091905
2021091906
2021091907
2021091908
2021091909
2021091910
2021091911
2021091912
2021091913
2021092000
2021092300
2021092600
2021092601
2021092602
2021092603
2021092604
2021092605
2021092606
2021092607
2021092608
2021092609
2021092610
2021092611
2021092612
2021092613
2021092700
2021093000
2021100300
2021100301
2021100302
2021100303
2021100304
2021100305
2021100306
2021100307
2021100308
2021100309
2021100310
2021100311
2021100312
2021100313
2021100400
2021100700
2021101000
2021101001
2021101002
2021101003
2021101004
2021101005
2021101006
2021101007
2021101008
2021101009
2021101010
2021101011
2021101012
2021101013
2021101100
2021101400
2021101700
2021101701
2021101702
2021101703
2021101704
2021101705
2021101706
2021101707
2021101708
2021101709

In [46]:
#groups down each game and play to frame to have it ready for adding data we 
# calculated above
grouped_by_frame = data[['gameId','playId','frameId']].groupby(['gameId','playId','frameId'])
grouped_down = grouped_by_frame.mean().reset_index()

In [47]:
#adds in data created above
grouped_down['a']= a
grouped_down['b']= b
grouped_down['c']= c
grouped_down['is_good_pocket']= is_good_pocket
grouped_down['frame_last_good_pocket']= frame_last_good_pocket
grouped_down['frame_from_snap']= frame_from_snap
grouped_down['ball_thrown_frame'] = ball_thrown_frame

In [48]:
grouped_down.to_csv("all_weeks_processed_final.csv")